In [2]:
import pandas as pd
import numpy as np

In [3]:
de = pd.read_csv('../differential_expression_analysis/output/edgr_exclude_one/intersect.tsv', sep='\t')
de = de.set_index('ensembl', drop=False)

mapp = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)

de_keep = pd.read_csv('../differential_expression_analysis/output/edgr_exclude_one/keep.intersect.txt', header=None)
de_keep = de_keep[0].to_list()
de_keep = mapp.loc[de_keep]['Description'].to_list()

rel = pd.read_csv('../data/family_summaries.tsv', sep='\t')
rel = rel.set_index('subject', drop=False)
rel = rel.drop('SG011')

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# format network df

In [4]:
net = pd.read_csv('all_gene_network_simple.tsv', sep='\t')

net.columns = ['startg', 'endg', 'distance',
       'num_connector_genes']

net = net.set_index(['startg', 'endg'], drop=False)

net = net.rename_axis((None, None))

# create df to hold output statistics

In [36]:
net_genes = net.startg.unique()
de_genes = de.gene.to_list()

de_and_net_genes = [s for s in de_genes if s in net_genes]

In [38]:
df = pd.DataFrame(index=de_and_net_genes)
df['avg_shortest_distance'] = 0.0

In [39]:
subnet = net[net.endg.isin(de_and_net_genes)].copy()

In [40]:
subnet = subnet[subnet.endg.isin(de_and_net_genes)].copy()


In [44]:

for gene in de_and_net_genes:
    subsubnet = subnet[subnet.startg == gene]
    subsubnet = subsubnet[subsubnet.endg != gene]
    df.at[gene, 'avg_shortest_distance'] = subsubnet.num_connector_genes.mean()

In [49]:
df.to_csv('gde2gde.tsv', sep='\t', index=False)